In [1]:
import tensorflow as tf
import numpy as np
import efficientnet.tfkeras as efn

from tensorflow.keras import layers, optimizers, datasets, Sequential, losses
from tensorflow.keras.layers import Dense, Flatten, Conv2D, GlobalAveragePooling2D
from sklearn.utils import shuffle

def load_dataset():
    # 在线下载，加载 CIFAR10 数据集
    (X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
    # 删除 y 的一个维度， [b,1] => [b]
    y_train = tf.squeeze(y_train, axis=1)
    y_test = tf.squeeze(y_test, axis=1)
    # 构建训练集对象，随机打乱，预处理，批量化
    """
    tf.data.Dataset.from_tensor_slices()的输入可以是numpy也可以是tensor，如果是numpy会自动转化为tensor。
    tf.data.Dataset.shuffle()函数的作用是打乱数据，参数为缓冲区的数据条数
    map表示预处理,参数为与处理函数
    """
    train_db = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_db = train_db.shuffle(1000).map(preprocess).batch(128)
    # 构建测试集对象，预处理，批量化
    test_db = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    test_db = test_db.map(preprocess).batch(64)
    # 从训练集中采样一个 Batch，并观察
    sample = next(iter(train_db))
    #print('sample:', sample[0].shape, sample[1].shape, tf.reduce_min(sample[0]), tf.reduce_max(sample[0]))
    return train_db, test_db

def preprocess(x, y):
    x = 2 * tf.cast(x, dtype=tf.float32) / 255. - 1
    y = tf.cast(y, dtype=tf.int32)
    return x, y

def load_dataset1():
    # 在线下载，加载 CIFAR10 数据集
    (X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
    # 删除 y 的一个维度， [b,1] => [b]
    #y_train = tf.squeeze(y_train, axis=1)
    #y_test = tf.squeeze(y_test, axis=1)
    #(X_train, y_train) = shuffle((X_train, y_train))
    X_train, y_train = preprocess(X_train, y_train)
    X_test, y_test = preprocess(X_test, y_test)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    return X_train, y_train, X_test, y_test

#train_db, test_db = load_dataset()
X_train, y_train, X_test, y_test = load_dataset1()
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 10)
(10000, 10)


In [2]:
def Model():
    baseline = efn.EfficientNetB7(input_shape=(32,32,3), weights='imagenet', include_top=False)
    model = Sequential()
    model.add(baseline)
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())###把上层的输出拉平
    model.add(Dense(10, activation="softmax"))
    return model

In [3]:
model = Model()
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=optimizers.Adam(), metrics=['accuracy'])

In [4]:
model.fit(X_train, y_train, batch_size=128, epochs=15, validation_data=(X_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 143s 3ms/sample - loss: 1.0585 - accuracy: 0.6354 - val_loss: 0.6953 - val_accuracy: 0.7889
Epoch 2/15
50000/50000 [==============================] - 105s 2ms/sample - loss: 0.5193 - accuracy: 0.8287 - val_loss: 0.5445 - val_accuracy: 0.8287
Epoch 3/15
50000/50000 [==============================] - 105s 2ms/sample - loss: 0.3741 - accuracy: 0.8764 - val_loss: 0.4707 - val_accuracy: 0.8487
Epoch 4/15
50000/50000 [==============================] - 105s 2ms/sample - loss: 0.3045 - accuracy: 0.8999 - val_loss: 0.4714 - val_accuracy: 0.8565
Epoch 5/15
50000/50000 [==============================] - 105s 2ms/sample - loss: 0.2380 - accuracy: 0.9193 - val_loss: 0.5079 - val_accuracy: 0.8508
Epoch 6/15
50000/50000 [==============================] - 105s 2ms/sample - loss: 0.1813 - accuracy: 0.9394 - val_loss: 0.5453 - val_accuracy: 0.8538
Epoch 7/15
50000/50000 [==========================